# View 3D images using napari

In [2]:
import numpy as np
import pandas as pd
import tifffile
import napari
from scipy.ndimage import label

In [3]:
# Load the segmentation mask
mask_file = "../data/Kuett_2022/MainHer2BreastCancerModel/measured_mask_final_segmentation_hwatershed_500.00_90%.tif"

segmentation_mask = tifffile.imread(mask_file)
segmentation_mask = segmentation_mask.astype(np.uint32) ## Needs to match cell id type
segmentation_mask.shape

(152, 488, 652)

In [ ]:
# Optional: View full segmentation mask
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(
    segmentation_mask>0, # binarize
    name='segmentation_mask', 
    scale=[2,1,1], # z-axis scaling
    colormap='gray', 
    blending='minimum', 
    contrast_limits=(0, 1), 
    rendering='attenuated_mip',
    opacity=0.5
)
viewer.dims.order = (0,1,2) # Adjust as needed
                            # use (1,0,2) to display z axis on y axis ('height')
napari.run()

/home/gesa/.py_venv/napari/lib/python3.12/site-packages/napari/plugins/_plugin_manager.py:555: UserWarning: Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten
  warn(message=warn_message)


In [23]:
# Load the phenotypes table
categories_file = '../data/Kuett_2022/MainHer2BreastCancerModel/model201710_cluster_labels_phenograph_recoded.csv'
cell_id_col = 'id' ## column with cell IDs
def load_phenotype(file: str, cell_id_col: str, categories_col: str) -> pd.DataFrame:
    phenotypes = pd.read_csv(file)
    phenotypes = phenotypes.rename(columns={cell_id_col: 'id', categories_col: 'phenotype'})
    phenotypes['id'] = phenotypes['id'].astype(np.uint32) ## needs to match cell id type
    return phenotypes

In [18]:
## Parameters to view 'pattern1' in Kuett et al. (2022):
# "we observed a clear tendency of CD45+CD3+ T cells (cluster 13 and 18) 
# to cluster around the vWF+CD31+ endothelial cells (cluster 21)"
categories_col = 'phenograph'
categories = load_phenotype(categories_file, cell_id_col, categories_col)
categories['phenotype'] = categories['phenotype'].astype(str)
categories.loc[ (categories['phenotype']=='13') | (categories['phenotype']=='18'), 'phenotype' ] = 'T cell' # rename
categories.loc[ categories['phenotype']=='21' , 'phenotype'] = 'Endothelial cell'
cats_to_show = ['T cell', 'Endothelial cell']

In [15]:
## Parameters to view 'pattern2' in Kuett et al. (2022):
# "we observed spatial separation of different 
# subsets of CD68+ cells (clusters 30 and 28)"
categories_col = 'phenograph'
categories = load_phenotype(categories_file, cell_id_col, categories_col)
categories['phenotype'] = categories['phenotype'].astype(str)
categories.loc[ categories['phenotype']=='30', 'phenotype' ] = 'subtype A' # rename
categories.loc[ categories['phenotype']=='28', 'phenotype' ] = 'subtype B'
cats_to_show = ['subtype A', 'subtype B']

In [24]:
## Custom parameters
categories_col = 'ct_broad' # which column to use for categories (phenotypes)
categories = load_phenotype(categories_file, cell_id_col, categories_col)

# Select categories to show
#cats_to_show = categories['category'].unique() # all
cats_to_show = [ # modify list to show only selected categories
    'cancer cell',
    'stromal cell',
    'myofibroblast'
]

In [25]:
# Map categories to cell IDs
category_to_cell_ids = {
    category: categories.loc[categories['phenotype'] == category, 'id'].values
    for category in cats_to_show
}

In [ ]:
# Prepare Napari viewer
viewer = napari.Viewer(ndisplay=3)

# Function to extract category-specific regions efficiently
def extract_category_mask(segmentation, cell_ids, chunk_size=(50, 100, 100)):
    """
    Extracts a binary mask for the given cell IDs from the segmentation.
    Processes the mask in chunks to save memory.
    """
    binary_mask = np.zeros_like(segmentation, dtype=np.uint8)
    z_size, y_size, x_size = segmentation.shape

    for z_start in range(0, z_size, chunk_size[0]):
        for y_start in range(0, y_size, chunk_size[1]):
            for x_start in range(0, x_size, chunk_size[2]):
                # Define chunk boundaries
                z_end = min(z_start + chunk_size[0], z_size)
                y_end = min(y_start + chunk_size[1], y_size)
                x_end = min(x_start + chunk_size[2], x_size)

                # Extract chunk
                chunk = segmentation[z_start:z_end, y_start:y_end, x_start:x_end]

                # Identify relevant pixels
                chunk_mask = np.isin(chunk, cell_ids)
                binary_mask[z_start:z_end, y_start:y_end, x_start:x_end] = chunk_mask

    return binary_mask

# Process and add each category
for category, cell_ids in category_to_cell_ids.items():
    mask = extract_category_mask(segmentation_mask, cell_ids)
    viewer.add_image(                                           # Adjust display as needed
        mask, 
        name=category,
        scale=[2,1,1], # z-axis scaling
        colormap='blue', 
        blending='minimum', 
        contrast_limits=(0, 1), 
        rendering='attenuated_mip',
        opacity=0.5
    )

viewer.dims.order = (0,1,2) # Adjust as needed
                            # use (1,0,2) to display z axis on y axis ('height')
napari.run()


/home/gesa/.py_venv/napari/lib/python3.12/site-packages/napari/plugins/_plugin_manager.py:555: UserWarning: Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten
  warn(message=warn_message)
